In [1]:
import psycopg2

def make_connection():

    host = 'localhost'  
    database = 'secure environment'
    user = 'postgres'
    password = '*****'
    port = '5432'  

    try:
        # Establish a connection to the database
        connection = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            port=port
        )
    
        # Create a cursor object to interact with the database
        cursor = connection.cursor()

        # Execute a sample query
        cursor.execute('SELECT version();')
        db_version = cursor.fetchone()
        print(f"Connected to: {db_version[0]}")

    except Exception as e:
        print(f"Error: {e}")
    return connection

   

In [2]:
import cv2
import numpy as np
from PIL import Image
import ast


In [3]:
from keras_facenet import FaceNet

face_model = FaceNet()


In [4]:
from ultralytics import YOLO
yolo_model = YOLO("yolov8n.pt")

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to compare face embeddings (cosine similarity)
def get_similarity(embedding1, embedding2):
    embedding1 = np.reshape(embedding1, (1, -1))  
    embedding2 = np.reshape(embedding2, (1, -1)) 
    similarity = cosine_similarity(embedding1, embedding2)[0][0]  # Get the similarity value
    return similarity 

In [6]:

# Assuming these functions and models are already initialized
model = yolo_model
con = make_connection()

# Open a connection to the webcam 
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Cannot access the webcam")
    exit()

# Function to pad the embedding
def pad_embedding(embedding, target_size=512):
    embedding = np.array(embedding, dtype=np.float32)
    current_size = len(embedding)
    
    if current_size > target_size:
        return embedding[:target_size]  
    elif current_size < target_size:
        padding = np.zeros(target_size - current_size, dtype=np.float32)
        return np.concatenate((embedding, padding))
    return embedding

# Function to get the face embedding
def get_face_embedding(image, target_size=512):
    face_image_pil = Image.fromarray(image).resize((160, 160))  
    face_image_array = np.array(face_image_pil)
    embeddings = face_model.embeddings([face_image_array])[0]
    
    padded_embedding = pad_embedding(embeddings, target_size)
    return padded_embedding

# Process frames from the webcam in real-time
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Perform face and object detection
    results = model(frame)
    
    cursor = con.cursor()
    cursor.execute("SELECT name, emb_vector FROM face_embedding")
    rows = cursor.fetchall()

    cell_phone_detected = False

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls.item())
        confidence = box.conf.item()
        label = model.names[class_id]

        # If a cell phone is detected, set the flag
        if class_id == 67:
            cell_phone_detected = True
            cv2.putText(frame, "Using cell phone", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # If no cell phone is detected, proceed with face recognition
    if not cell_phone_detected:
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls.item())
            confidence = box.conf.item()

            if class_id == 0 and confidence > 0.7:  # Face class
                person_reg = frame[y1:y2, x1:x2]
                face_roi = person_reg[0:(y2 - y1)//2, :]  

                embedded_face = get_face_embedding(face_roi, target_size=512)

                for row in rows:
                    embedding = row[1]

                    

                    similarity = get_similarity(embedded_face, np.array(embedding))
                    
                    if similarity > 0.7:
                        color = (0, 255, 0)
                        name = row[0]
                        print(f"Recognized: {name}")
                        break
                else:
                    name = "Unknown"
                    color = (0, 0, 255)
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f'{name}: {confidence:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the output frame
    cv2.imshow('YOLOv8 Real-Time Face Detection', frame)

    # Break on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Connected to: PostgreSQL 16.3, compiled by Visual C++ build 1939, 64-bit

0: 480x640 2 persons, 535.2ms
Speed: 43.3ms preprocess, 535.2ms inference, 39.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step

0: 480x640 2 persons, 358.9ms
Speed: 45.4ms preprocess, 358.9ms inference, 46.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step

0: 480x640 2 persons, 327.0ms
Speed: 9.5ms preprocess, 327.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step

0: 480x640 2 persons, 1 tv, 167.6ms
Speed: 4.0ms preprocess, 167.6ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step

0: 480x640 2 persons, 162.5ms
Speed: 3.0ms preprocess, 162.5ms inference, 4.4ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step

0: 480x640 2 persons, 191.5ms
Speed: 3.4ms preprocess, 191.5ms inference, 3.0ms